In [1]:
from dotenv import load_dotenv

assert load_dotenv("../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev17+gfcf7439'

In [3]:
from ragna import demo_config, Config

demo_config

Config(local_cache_root=PosixPath('/home/philip/.cache/ragna'), state_database_url='sqlite:////home/philip/.cache/ragna/ragna.db', queue_database_url='redis://127.0.0.1:6379', registered_source_storage_classes={'Ragna/DemoSourceStorage': <class 'ragna.source_storage._demo.RagnaDemoSourceStorage'>}, registered_llm_classes={'Ragna/DemoLLM': <class 'ragna.llm._demo.RagnaDemoLlm'>})

In [4]:
from pathlib import Path

document_path = Path.cwd() / "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.core import Rag
from ragna.llm import RagnaDemoLlm
from ragna.source_storage import RagnaDemoSourceStorage
from ragna.llm import OpenaiGpt35Turbo16kLlm, OpenaiGpt4Llm
from ragna.source_storage import ChromaSourceStorage

rag = Rag(demo_config)

prompt = "What is Ragna?"

async with await rag.new_chat(
    documents=[document_path],
    source_storage=ChromaSourceStorage,
    llm=OpenaiGpt35Turbo16kLlm,
) as chat:
    print(await chat.answer(prompt))

In [15]:
import asyncio
import itertools
import time
from pprint import pprint

from ragna.llm import OpenaiGpt35Turbo16kLlm, OpenaiGpt4Llm
from ragna.source_storage import ChromaSourceStorage

source_storages = [ChromaSourceStorage]
llms = [OpenaiGpt35Turbo16kLlm, OpenaiGpt4Llm]


async def answer_prompt(source_storage, llm):
    start = time.time()
    async with await rag.new_chat(
        documents=[document_path],
        source_storage=source_storage,
        llm=llm,
    ) as chat:
        answer = await chat.answer(prompt)
    print(
        f"{source_storage.display_name()} / {llm.display_name()}: {time.time() - start:.1f}s"
    )
    return answer


experiments = {
    (source_storage.display_name(), llm.display_name()): answer_prompt(
        source_storage, llm
    )
    for source_storage, llm in itertools.product(source_storages, llms)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7faa4f393b40>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7faa4f393a40>}


In [16]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))

pprint(results)

Chroma / OpenAI/gpt-3.5-turbo-16k: 5.8s
Chroma / OpenAI/gpt-4: 10.0s
{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): 'Ragna is an open-source RAG '
                                         'orchestration app. It is designed to '
                                         'help users manage and automate their '
                                         'RAG (Red, Amber, Green) status '
                                         'reporting process. Ragna allows '
                                         'users to easily track and update the '
                                         'status of various tasks or projects, '
                                         'and provides a visual representation '
                                         'of the overall progress. It is a '
                                         'useful tool for teams or individuals '
                                         'who need to regularly report on the '
                                         'status of their work.',